In [65]:
#!pip3 freeze > requirements.txt

In [2]:
!pip install selenium
!pip install scrapy
!pip install typing-extensions==4.3.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 957.9 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 7.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.3/158.3 kB 3.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.2/400.2 kB 9.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 14.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 14.1 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.9/83.9 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [50]:
import numpy as np
import pandas as pd
import math
from scrapy.selector import Selector
from selenium import webdriver 
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from tqdm import tqdm
import warnings
import os
warnings.filterwarnings("ignore")

ModuleNotFoundError: No module named 'scrapy'

In [7]:
filepath = open('../data/raw/urls/imdb.txt', 'r')
lines = filepath.readlines()

In [8]:
for url in lines:
    url = url+'/reviews'
    driver = webdriver.Chrome()
    time.sleep(1)
    driver.get(url)
    # time.sleep(1)
    movie_name = driver.title.split(')')[0]
    print(movie_name)
    if movie_name in os.listdir('../data/raw/imdb_reviews/'):
        pass
    # time.sleep(1)

    body = driver.find_element(By.CSS_SELECTOR, 'body')
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)
    body.send_keys(Keys.PAGE_DOWN)

    sel = Selector(text = driver.page_source)
    review_counts = sel.css('.lister .header span::text').extract_first().replace(',','').split(' ')[0]
    print(review_counts)
    more_review_pages = int(int(review_counts)/25)

    for i in tqdm(range(more_review_pages)):
        try:
            # body.send_keys(Keys.PAGE_DOWN)
            # time.sleep(1)
            # body.send_keys(Keys.PAGE_DOWN)
            # time.sleep(1)
            # body.send_keys(Keys.PAGE_DOWN)
            # time.sleep(1)
            # body.send_keys(Keys.PAGE_DOWN)
            # time.sleep(1)
            driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
            time.sleep(3)
            css_selector = 'load-more-trigger'
            driver.find_element(By.ID, css_selector).click()
            # time.sleep(0.5)
        except:
            pass
    
    # review = sel2.css('.text.show-more__control::text').extract_first().strip()
    # review_date = sel2.css('.review-date::text').extract_first().strip()
    # author = sel2.css('.display-name-link a::text').extract_first().strip()
    # review_title = sel2.css('a.title::text').extract_first().strip()
    # review_url = sel2.css('a.title::attr(href)').extract_first().strip()
    # helpfulness = sel2.css('.actions.text-muted::text').extract_first().strip()
    # print('nRating:',rating)
    # print('nreview_title:',review_title)
    # print('nAuthor:',author)
    # print('nreview_date:',review_date)
    # print('nreview:',review)
    # print('nhelpfulness:',helpfulness)


    rating_list = []
    review_date_list = []
    review_title_list = []
    author_list = []
    review_list = []
    review_url_list = []
    error_url_list = []
    error_msg_list = []
    reviews = driver.find_elements(By.CSS_SELECTOR, 'div.review-container')

    for d in tqdm(reviews):
        try:
            sel2 = Selector(text = d.get_attribute('innerHTML'))
            try:
                rating = sel2.css('.rating-other-user-rating span::text').extract_first()
            except:
                rating = np.NaN
            try:
                review = sel2.css('.text.show-more__control::text').extract_first()
            except:
                review = np.NaN
            try:
                review_date = sel2.css('.review-date::text').extract_first()
            except:
                review_date = np.NaN    
            try:
                author = sel2.css('.display-name-link a::text').extract_first()
            except:
                author = np.NaN    
            try:
                review_title = sel2.css('a.title::text').extract_first()
            except:
                review_title = np.NaN
            try:
                review_url = sel2.css('a.title::attr(href)').extract_first()
            except:
                review_url = np.NaN
            rating_list.append(rating)
            review_date_list.append(review_date)
            review_title_list.append(review_title)
            author_list.append(author)
            review_list.append(review)
            review_url_list.append(review_url)
        except Exception as e:
            error_url_list.append(url)
            error_msg_list.append(e)
    movie = [movie_name for i in range(len(review_date_list))]
    review_df = pd.DataFrame({
        'Review_Date':review_date_list,
        'Author':author_list,
        'Rating':rating_list,
        'Review_Title':review_title_list,
        'Review':review_list,
        'Review_Url':review_url,
        'Movie' : movie
        })
    review_df.to_csv(f'../data/raw/imdb_reviews/{movie_name}.csv')
    # break

In [58]:
df = pd.read_csv('../data/raw/imdb_reviews/combined.csv').drop(columns=['Unnamed: 0'])
df['Movie'] = df.Movie.apply(lambda x: x.split('_')[:-1])
df['Movie'] = df.Movie.apply(lambda x: '-'.join(x))
def antman(movie):
    if movie == 'AntMan':
        return 'ant-man'
    else:
        return movie.lower()
df['Movie'] = df.Movie.apply(lambda x: antman(x))


In [59]:
names = pd.read_json('../data/raw/imdb_reviews/mcu_list.json')

In [60]:
df = df.rename(columns={
    'Review_Date' : 'date',
    'Author' : 'user',
    'Rating' : 'score',
    'Movie' : 'movie',
    'Review' : 'text',
    'Review_Title' : 'title',
    'Review_Url' : 'url'
})
df['date'] = pd.to_datetime(df['date'])

def score(num):
    if not math.isnan(num):
        return int(num*10)
    else:
        return num
df['score'] = df.score.apply(lambda x: score(x))

In [64]:
df.to_parquet('../data/raw/imdb_reviews/cleaned_sorta.parquet')

In [63]:
!pip install pyarrow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.8/25.8 MB 8.6 MB/s eta 0:00:0000:0100:01


In [26]:
sorted(names)

['ant-man',
 'ant-man-and-the-wasp',
 'ant-man-and-the-wasp-quantumania',
 'avengers-age-of-ultron',
 'avengers-endgame',
 'avengers-infinity-war',
 'black-panther',
 'black-panther-wakanda-forever',
 'black-widow',
 'captain-america-civil-war',
 'captain-america-the-first-avenger',
 'captain-america-the-winter-soldier',
 'captain-marvel',
 'doctor-strange',
 'doctor-strange-in-the-multiverse-of-madness',
 'eternals',
 'guardians-of-the-galaxy',
 'guardians-of-the-galaxy-vol.-2',
 'guardians-of-the-galaxy-vol.-3',
 'iron-man',
 'iron-man-2',
 'iron-man-3',
 'shang-chi-and-the-legend-of-the-ten-rings',
 'spider-man-far-from-home',
 'spider-man-homecoming',
 'spider-man-no-way-home',
 'the-avengers',
 'the-guardians-of-the-galaxy-holiday-special',
 'the-incredible-hulk',
 'thor',
 'thor-love-and-thunder',
 'thor-ragnarok',
 'thor-the-dark-world',
 'werewolf-by-night']

In [25]:
sorted(set(df.Movie))

['ant-man',
 'antman-and-the-wasp',
 'antman-and-the-wasp-quantumania',
 'avengers-age-of-ultron',
 'avengers-assemble',
 'avengers-endgame',
 'avengers-infinity-war',
 'black-panther',
 'black-panther-wakanda-forever',
 'black-widow',
 'captain-america-civil-war',
 'captain-america-the-first-avenger',
 'captain-america-the-winter-soldier',
 'captain-marvel',
 'doctor-strange',
 'doctor-strange-in-the-multiverse-of-madness',
 'eternals',
 'guardians-of-the-galaxy',
 'guardians-of-the-galaxy-vol-2',
 'guardians-of-the-galaxy-vol-3',
 'iron-man',
 'iron-man-2',
 'iron-man-3',
 'shangchi-and-the-legend-of-the-ten-rings',
 'spiderman-far-from-home',
 'spiderman-homecoming',
 'spiderman-no-way-home',
 'the-incredible-hulk',
 'thor',
 'thor-love-and-thunder',
 'thor-ragnarok',
 'thor-the-dark-world']